# Port Statistics

This notebook develops and explores the various port statistics used in the [Port Performance Project](https://github.com/epistemetrica/Port-Performance-Project). See the README.md file in the main directory for more info.

The primary data set comes from a combination of AIS vessel data and port data, processed in the Port Geodata notebook.

Statistics and final dataframes developed here are used in the Port Performance Dashboard.



In [89]:
#prelims
import polars as pl
import polars.selectors as cs
import pandas as pd
import geopandas as gpd
import time
import plotly.express as px
import matplotlib.pyplot as plt
import contextily as cx
import numpy as np
import glob
import folium
from folium.plugins import HeatMap

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=100);

## Load Data from geodata_prep notebook

In [90]:
#load data from parquet
main_lf = pl.scan_parquet('port data/dashboard/main.parquet')
#inspect
display(main_lf.describe())
main_lf.limit(5).collect()

statistic,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,which_port_waters,port_waters1,port_waters2,port_waters3,status_duration,dock_id,dock_name,facility_type,port_name,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon,call_id
str,str,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,str
"""count""","""1024175""","""1024175""",1.024175e6,1.021926e6,1.013653e6,1.024175e6,"""1024175""",1.024175e6,1.024167e6,1.024175e6,954103.0,952090.0,819386.0,"""1024175""","""1024175""","""1024175""","""1024175""","""1021344""","""1024175""","""1024175""","""1021198""","""1024175""",1.024175e6,1.024175e6,1.024175e6,1.024175e6,1.024175e6,1.024175e6,"""1024175"""
"""null_count""","""0""","""0""",0.0,2249.0,10522.0,0.0,"""0""",0.0,8.0,0.0,70072.0,72085.0,204789.0,"""0""","""0""","""0""","""0""","""2831""","""0""","""0""","""2977""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",null,"""2021-07-21 14:06:02.223104""",2.986939,184.897959,186.772071,1.768427,null,73.284826,1.0448e7,207.064062,31.613459,10.66267,74.018206,null,null,null,null,"""17:09:03.097916""",null,null,null,null,32.013095,-95.717791,31.974159,-95.897874,31.973375,-95.89908,null
"""std""",null,null,5.500858,96.188133,104.364344,2.394977,null,4.911119,3.1025e7,57.132527,7.462199,2.864933,7.911821,null,null,null,null,null,null,null,null,null,6.544798,20.08293,6.501684,19.909045,6.501389,19.908524,null
"""min""","""205042000""","""2018-01-01 01:14:33""",0.0,0.0,0.0,0.0,null,70.0,0.0,101.0,0.0,-12.8,0.0,"""Albany Port District, NY""","""Albany Port District, NY""","""Guaynabo, PR""","""Not in port waters""","""0:00:01""","""00XE""","""ADM Corpus Christi Grain Eleva…","""Anchorage""","""Albany Port District, NY""",12.37642,-170.62387,17.936081,-166.53444,17.938939,-166.549916,"""205042000_Port of Long Beach, …"
"""25%""",null,"""2019-11-10 00:07:14""",0.0,118.5,93.0,0.0,null,70.0,9.313058e6,179.0,27.0,8.7,70.0,null,null,null,null,"""1:18:01""",null,null,null,null,28.57496,-118.20663,28.645767,-118.21277,28.629389,-118.2095,null
"""50%""",null,"""2021-09-23 22:08:26""",0.1,185.2,183.0,0.0,null,70.0,9.462706e6,189.0,32.0,10.5,71.0,null,null,null,null,"""2:41:00""",null,null,null,null,30.21498,-93.84361,30.35667,-93.941127,30.359621,-93.96069,null
"""75%""",null,"""2023-03-20 11:10:25""",3.5,261.8,273.0,5.0,null,80.0,9.682863e6,229.0,32.0,12.8,80.0,null,null,null,null,"""7:45:41""",null,null,null,null,34.34973,-80.11536,34.201663,-80.122222,34.227365,-80.117801,null
"""max""","""725019340""","""2024-09-30 23:22:34""",102.3,359.9,359.0,15.0,null,89.0,9.901843e8,667.0,86.0,25.5,191.0,"""Yabucoa, PR""","""Yabucoa, PR""","""Yabucoa, PR""","""Yabucoa, PR""","""2191 days, 16:27:17""","""1JHK""","""YUSEN TERMINALS BERTHS 212-221""","""Tie Off""","""Wilmington, NC""",61.24418,144.67134,61.24306,-66.086926,61.23778,-66.096678,"""725019340_Corpus Christi, TX_2…"


mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,which_port_waters,port_waters1,port_waters2,port_waters3,status_duration,dock_id,dock_name,facility_type,port_name,vessel_lat,vessel_lon,dock_lat,dock_lon,port_lat,port_lon,call_id
str,datetime[μs],f64,f64,f64,f64,cat,f64,f64,f64,f64,f64,f64,str,str,str,str,duration[μs],str,str,str,str,f64,f64,f64,f64,f64,f64,str
"""205042000""",2022-12-07 23:20:37,9.1,358.1,0.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",2h 34m 41s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.36623,-118.04241,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-08 01:55:18,0.3,83.8,328.0,1.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",6d 18h 3m 47s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.62433,-118.0504,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-14 19:59:05,0.9,232.6,228.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",4h 35m 49s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.62758,-118.05079,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-15 00:34:54,0.0,340.4,340.0,5.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",1d 8h 25m 12s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.75624,-118.21814,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"
"""205042000""",2022-12-16 09:00:06,0.2,81.3,339.0,0.0,"""DELOS""",80.0,9.877767e6,336.0,60.0,16.5,80.0,"""Port of Long Beach, CA|Port of…","""Port of Long Beach, CA""","""Port of Los Angeles, CA""","""Not in port waters""",1h 2m 19s,"""0V0U""","""ARCO WESTERN PIPELINE CO BERTH…","""Dock""","""Port of Long Beach, CA""",33.75624,-118.21808,33.757222,-118.21888,33.73957,-118.2095,"""205042000_Port of Long Beach, …"


## Generate Stats

In [91]:
#get stats for each call
calls_df = (
    main_lf
    #ensure sorting by vessel and time
    .sort(['mmsi', 'time'])
    #grouby by call id
    .group_by('call_id')
    .agg(
        #port name
        port_name = pl.first('port_name'),
        #dock name
        dock_name = pl.first('dock_name'),
        #dock_id
        dock_id = pl.first('dock_id'),
        #vessel mmsi
        mmsi = pl.first('mmsi'),
        #vessel imo
        imo = pl.first('imo'),
        #vessel size
        vessel_size = pl.first('length'),
        #time entering port waters
        time_port_entry = pl.col('time').min(),
        #time of arrival at dock
        time_arrival = (
            pl.when(pl.col('status')==5)
            .then(pl.col('time'))
            .otherwise(pl.lit(None))
        ).min(),
        #time of departure from dock
        time_departure = (
            pl.when(pl.col('status')==5)
            .then(pl.col('time')+pl.col('status_duration'))
            .otherwise(pl.lit(None))
        ).max(),
        #time port exit
        time_port_exit = (pl.col('time') + pl.col('status_duration')).max(),
        #hrs a berth
        hrs_at_berth = (
            pl.when(pl.col('status')==5)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
            .sum().dt.total_minutes()/60
        ),
        #hrs at anchor
        hrs_at_anchor = (
            pl.when(pl.col('status')==1)
            .then(pl.col('status_duration'))
            .otherwise(pl.lit(None))
            .sum().dt.total_minutes()/60
        )
    )
    #compute additional stats
    .with_columns(
        #time from port entry to docking in hrs
        hrs_to_dock = (
            (pl.col('time_arrival') - pl.col('time_port_entry'))
            .dt.total_minutes()/60
        ),
        #time in port waters after leaving dock
        hrs_in_port_after_dock = (
            (pl.col('time_port_exit') - pl.col('time_departure'))
            .dt.total_minutes()/60
        ),
        #total time in port waters in hrs
        hrs_in_port_waters =(
            (pl.col('time_port_exit') - pl.col('time_port_entry'))
            .dt.total_minutes()/60
        )
    )
    #collect
    .collect()
)

#inspect
display(calls_df.describe())
calls_df.head(5)


statistic,call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64
"""count""","""155558""","""155558""","""155558""","""155558""","""155558""",155556.0,155558.0,"""155558""","""155558""","""155266""","""155509""",155558.0,155558.0,155558.0,155266.0,155509.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",2.0,0.0,"""0""","""0""","""292""","""49""",0.0,0.0,0.0,292.0,49.0
"""mean""",null,null,null,null,null,1.0139e7,209.068225,"""2021-06-08 09:08:02.359814""","""2021-06-09 13:46:17.326894""","""2021-06-13 06:00:57.885036""","""2021-06-14 23:32:36.168221""",69.889625,14.35373,28.629654,28.538581,159.978917
"""std""",null,null,null,null,null,2.6462e7,59.507905,null,null,null,null,571.122601,169.387675,641.291639,837.937791,1610.579183
"""min""","""205042000_Port of Long Beach, …","""Albany Port District, NY""","""ADM Corpus Christi Grain Eleva…","""00XE""","""205042000""",0.0,101.0,"""2018-01-01 01:14:33""","""2018-01-01 01:43:12""","""2018-01-01 04:23:54""","""2018-01-01 09:15:57""",0.0,0.0,0.0,0.0,0.0
"""25%""",null,null,null,null,null,9.298507e6,177.0,"""2019-08-28 23:09:30""","""2019-08-30 04:02:05""","""2019-09-01 23:05:51""","""2019-09-04 06:22:17""",16.683333,0.0,2.683333,2.1,25.683333
"""50%""",null,null,null,null,null,9.402316e6,191.0,"""2021-07-24 20:39:21""","""2021-07-26 14:02:25""","""2021-07-31 17:18:51""","""2021-08-03 03:31:22""",31.566667,0.0,3.633333,2.766667,44.283333
"""75%""",null,null,null,null,null,9.61385e6,233.0,"""2023-02-27 10:39:07""","""2023-02-28 10:58:32""","""2023-03-04 03:39:06""","""2023-03-06 20:02:05""",59.083333,0.0,5.866667,3.516667,84.466667
"""max""","""725019340_Corpus Christi, TX_2…","""Wilmington, NC""","""YUSEN TERMINALS BERTHS 212-221""","""1JHK""","""725019340""",9.901843e8,667.0,"""2024-09-30 12:54:53""","""2024-09-30 16:48:33""","""2024-09-30 23:22:34""","""2024-09-30 23:22:34""",58940.316667,52600.45,58343.5,58107.066667,59115.216667


call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],f64,f64,f64,f64,f64
"""563134100_Port Freeport, TX_20…","""Port Freeport, TX""","""Phillips 66 Co., Freeport Term…","""0SRW""","""563134100""",9.547506e6,179.0,2021-09-10 20:03:43,2021-09-11 00:28:42,2021-09-13 01:52:57,2021-09-13 05:28:28,49.4,0.0,4.4,3.583333,57.4
"""477308200_Port of Oakland, CA_…","""Port of Oakland, CA""","""SCHNITZER STEEL PRODUCTS, 7TH …","""0T48""","""477308200""",9.753296e6,180.0,2023-12-28 15:33:41,2024-01-02 16:08:09,2024-01-08 12:44:36,2024-01-08 15:44:57,139.233333,114.983333,120.566667,3.0,264.183333
"""212439000_Mobile, AL_2018-04-2…","""Mobile, AL""","""AGREX GRAIN ELEVATOR AND ALABA…","""0SE9""","""212439000""",9.300764e6,182.0,2018-04-24 06:41:30,2018-04-24 09:43:11,2018-05-04 22:02:30,2018-05-05 00:42:52,252.316667,0.0,3.016667,2.666667,258.016667
"""271042759_Virginia, VA, Port o…","""Virginia, VA, Port of""","""PORTSMOUTH MARINE TERMINAL WHA…","""0X4W""","""271042759""",9.365867e6,182.0,2019-07-08 14:10:00,2019-07-08 14:10:00,2019-07-08 22:05:28,2019-07-09 01:13:42,7.916667,0.0,0.0,3.133333,11.05
"""311056900_Port of Long Beach, …","""Port of Long Beach, CA""","""MANSON CONST & ENGINEERING CO …","""0VCP""","""311056900""",9.600994e6,228.0,2024-03-22 09:30:39,2024-03-22 14:52:31,2024-03-24 04:13:34,2024-03-25 20:56:07,37.35,37.25,5.35,40.7,83.416667


### Notes on Calls Frame and additional cleaning

The mean and quartile statistics for the port calls seems reasonable, but 0 values should not be observed for several stats (e.g., hrs_in_port_waters) since the geodata_prep notebook has already limited the data to vessels that dock at the relevant port. We are working to track down the source of the issue; however, the relevant observations are few and are dropped as outliers for now. 

A similar issue exists with very long hrs_at_berth and related stats. The reason is better understood, as this would result from vessel AIS transponders going offline at some stage during their visit to port waters. Rectifying this issue will be done either in the AIS ingestion or geodata_prep stages at a later date. For now we also drop these as outliers. 

In [92]:
#define zero values drop
def drop_zero_values(df, cols):
    '''
    Drops zero values from the dataframe for the specified columns.
    Args:
        df: Polars DataFrame
        cols: List of columns to drop zero values from
    Returns:
        Polars DataFrame with zero values dropped
    '''
    for col in cols:
        df = df.filter(pl.col(col) > 0)
    return df

#define outlier drop
def drop_outliers(df, cols, threshold=3):
    '''
    Drops outliers from the dataframe for the specified columns.
    Args:
        df: Polars DataFrame
        cols: List of columns to drop outliers from
        threshold: Z-score threshold for outlier detection
    Returns:
        Polars DataFrame with outliers dropped
    '''
    print(f'Outlier threshold: {threshold} Std Devs')
    for col in cols:
        #compute z scores
        df = df.with_columns(
            z_score = (pl.col(col) - pl.col(col).mean()) / pl.col(col).std()
        )
        #drop outliers
        df = df.filter(pl.col('z_score').abs() < threshold)
    return df.drop('z_score')

In [93]:
#get count of rows from calls_df before drop
rows_prior = calls_df.shape[0]

#list cols that should never be zero by construction
zero_cols = ['hrs_at_berth', 'hrs_to_dock', 'hrs_in_port_after_dock', 
             'hrs_in_port_waters']
#list cols for outlier drop
outlier_cols = ['hrs_at_berth', 'hrs_to_dock', 'hrs_at_anchor', 
                'hrs_in_port_after_dock', 'hrs_in_port_waters']

#drop zero values
calls_df = drop_zero_values(calls_df, zero_cols)
#print number of observations dropped
print(f'Total zero rows dropped: {rows_prior - calls_df.shape[0]} of {rows_prior}')
rows_prior = calls_df.shape[0]

#drop outliers
#print z_score thresholds
for col in outlier_cols:
    print(f'{col} outlier threshold: {calls_df[col].std()*3/24:.1f} days')
#drop outliers
calls_df = drop_outliers(calls_df, outlier_cols, threshold=3)
#print rows dropped
print(f'Total outlier rows dropped: {rows_prior - calls_df.shape[0]} of {rows_prior}')

#inspect
display(calls_df.describe())
calls_df.head()

Total zero rows dropped: 9687 of 155558
hrs_at_berth outlier threshold: 37.7 days
hrs_to_dock outlier threshold: 59.6 days
hrs_at_anchor outlier threshold: 20.2 days
hrs_in_port_after_dock outlier threshold: 88.5 days
hrs_in_port_waters outlier threshold: 143.6 days
Outlier threshold: 3 Std Devs
Total outlier rows dropped: 889 of 145871


statistic,call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,str,f64,f64,str,str,str,str,f64,f64,f64,f64,f64
"""count""","""144982""","""144982""","""144982""","""144982""","""144982""",144981.0,144982.0,"""144982""","""144982""","""144982""","""144982""",144982.0,144982.0,144982.0,144982.0,144982.0
"""null_count""","""0""","""0""","""0""","""0""","""0""",1.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,null,null,1.0134e7,208.722717,"""2021-05-31 15:26:13.730380""","""2021-06-01 05:18:12.087797""","""2021-06-03 08:54:25.962981""","""2021-06-03 14:23:30.308528""",47.328491,11.467013,13.858178,5.47655,70.946423
"""std""",null,null,null,null,null,2.6380e7,59.276192,null,null,null,null,54.00096,42.652892,39.556875,25.281921,82.264076
"""min""","""205042000_Port of Long Beach, …","""Albany Port District, NY""","""ADM Corpus Christi Grain Eleva…","""00XE""","""205042000""",0.0,101.0,"""2018-01-01 01:14:33""","""2018-01-01 04:11:35""","""2018-01-01 17:53:33""","""2018-01-01 22:14:43""",0.016667,0.0,0.016667,0.016667,1.966667
"""25%""",null,null,null,null,null,9.298351e6,176.0,"""2019-08-15 04:17:04""","""2019-08-15 16:14:11""","""2019-08-17 12:52:49""","""2019-08-17 17:53:35""",16.933333,0.0,2.75,2.2,25.966667
"""50%""",null,null,null,null,null,9.401805e6,190.0,"""2021-07-05 19:53:31""","""2021-07-06 13:27:06""","""2021-07-09 00:02:57""","""2021-07-09 05:28:58""",31.366667,0.0,3.666667,2.8,43.933333
"""75%""",null,null,null,null,null,9.612997e6,232.0,"""2023-02-25 09:33:03""","""2023-02-26 00:22:39""","""2023-02-28 02:45:37""","""2023-02-28 06:53:17""",57.65,0.0,5.95,3.55,81.916667
"""max""","""725019340_Corpus Christi, TX_2…","""Wilmington, NC""","""YUSEN TERMINALS BERTHS 212-221""","""1JHK""","""725019340""",9.800025e8,667.0,"""2024-09-30 07:46:57""","""2024-09-30 11:36:06""","""2024-09-30 22:03:53""","""2024-09-30 22:57:53""",954.933333,493.116667,1249.066667,1312.2,1405.483333


call_id,port_name,dock_name,dock_id,mmsi,imo,vessel_size,time_port_entry,time_arrival,time_departure,time_port_exit,hrs_at_berth,hrs_at_anchor,hrs_to_dock,hrs_in_port_after_dock,hrs_in_port_waters
str,str,str,str,str,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],f64,f64,f64,f64,f64
"""563134100_Port Freeport, TX_20…","""Port Freeport, TX""","""Phillips 66 Co., Freeport Term…","""0SRW""","""563134100""",9.547506e6,179.0,2021-09-10 20:03:43,2021-09-11 00:28:42,2021-09-13 01:52:57,2021-09-13 05:28:28,49.4,0.0,4.4,3.583333,57.4
"""477308200_Port of Oakland, CA_…","""Port of Oakland, CA""","""SCHNITZER STEEL PRODUCTS, 7TH …","""0T48""","""477308200""",9.753296e6,180.0,2023-12-28 15:33:41,2024-01-02 16:08:09,2024-01-08 12:44:36,2024-01-08 15:44:57,139.233333,114.983333,120.566667,3.0,264.183333
"""212439000_Mobile, AL_2018-04-2…","""Mobile, AL""","""AGREX GRAIN ELEVATOR AND ALABA…","""0SE9""","""212439000""",9.300764e6,182.0,2018-04-24 06:41:30,2018-04-24 09:43:11,2018-05-04 22:02:30,2018-05-05 00:42:52,252.316667,0.0,3.016667,2.666667,258.016667
"""311056900_Port of Long Beach, …","""Port of Long Beach, CA""","""MANSON CONST & ENGINEERING CO …","""0VCP""","""311056900""",9.600994e6,228.0,2024-03-22 09:30:39,2024-03-22 14:52:31,2024-03-24 04:13:34,2024-03-25 20:56:07,37.35,37.25,5.35,40.7,83.416667
"""309822000_Redwood City, CA_201…","""Redwood City, CA""","""PORT OF REDWOOD CITY WHARVES N…","""0VDL""","""309822000""",7.926174e6,244.0,2018-05-31 08:43:16,2018-06-01 11:26:43,2018-06-01 15:14:50,2018-06-01 18:57:59,3.8,8.033333,26.716667,3.716667,34.233333


### Time Awaiting Berth

We define time awaiting berth as the total time it takes a vessel to get to the dock minus the amount of time the dock was occupied while that vessel was en route. 

In [48]:
#calculate time awaiting berth

#for each call_id and dock, get the total time dock was occupied between time_port_entry and time_arrival

#get time_port_entry, time_arrival for each call id
lf = (calls_df.select('call_id', 'time_port_entry', 'time_arrival')
      .unique().lazy())
#join to main lf
main_lf = main_lf.join(lf, on='call_id', how='left')

for call in calls_df.select('call_id').unique().to_series():
      #get start time and end time
      start = (calls_df.filter(pl.col('call_id')==call)
               .select('time_port_entry').item())
      end = (calls_df.filter(pl.col('call_id')==call)
               .select('time_arrival').item())
      #get dock occupancy
      df = (
            main_lf
            .with_columns(
                  dock_occupied = (
                        (pl.col('status')==5)
                        .then(pl.col('status_duration'))
                        .otherwise(pl.lit(None))
                  )
            )
      )


AttributeError: 'Expr' object has no attribute 'then'

In [72]:
#create monthly stats dataframe
monththly_df = (
    calls_df
    #get month from docking time
    .with_columns(
        #extract month from docking time
        month = pl.col('time_arrival').dt.strftime('%Y%m')
    )
    #group by port dock and month
    .group_by(['port_name', 'dock_id', 'month'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_occupied = pl.sum('hrs_at_berth'),
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor'),
        #time in port waters 
        hrs_in_port_waters_total = pl.sum('hrs_in_port_waters'),
        hrs_in_port_waters_mean = pl.mean('hrs_in_port_waters'),
        hrs_in_port_waters_median = pl.median('hrs_in_port_waters')
    )
    #get hours from each month
    .with_columns(
        hrs_in_month = (
            pl.when(pl.col('month').str.tail(2).is_in(['01', '03', '05', '07',
                                                       '08', '10', '12']))
            .then(31*24)
            .when(pl.col('month').str.tail(2).is_in(['04', '06', '09', '11']))
            .then(30*24)
            .otherwise(28*24)
        )
    )
    .with_columns(
        #dock utilization - percentage of time a dock is occupied
        utilization = (
            pl.col('hrs_occupied')/pl.col('hrs_in_month')
        )
    )
    #drop hours in month
    .drop('hrs_in_month')
    #sort by port dock then month
    .sort(['port_name', 'dock_id', 'month'])
)

#inspect
display(monththly_df.describe())
monththly_df.head()

statistic,port_name,dock_id,month,vessels,vessel_size_mean,calls,hrs_occupied,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean,hrs_in_port_waters_total,hrs_in_port_waters_mean,hrs_in_port_waters_median,utilization
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""34196""","""34196""","""34196""",34196.0,34196.0,34196.0,34196.0,34196.0,34196.0,34196.0,34196.0,34196.0,34196.0,34196.0,34196.0
"""null_count""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,3.745701,196.443795,4.239736,200.660288,58.97395,61.628853,9.544923,12.349109,300.794079,87.693096,82.809905,0.27506
"""std""",null,null,null,3.860586,49.429054,4.58267,188.965615,60.669452,60.569896,33.738438,35.239439,328.046394,81.663593,81.245359,0.258927
"""min""","""Albany Port District, NY""","""00XE""","""201801""",1.0,101.0,1.0,0.033333,0.033333,0.033333,0.0,0.0,3.266667,3.266667,3.266667,0.00005
"""25%""",null,null,null,1.0,168.8,1.0,66.533333,23.108333,25.585714,0.0,0.0,95.5,37.445833,34.125,0.091263
"""50%""",null,null,null,3.0,185.0,3.0,145.883333,41.166667,44.276667,0.0,0.0,202.716667,63.472222,58.066667,0.200162
"""75%""",null,null,null,5.0,212.666667,5.0,276.05,72.95,76.619444,0.0,5.386667,386.0,108.683333,101.6,0.378786
"""max""","""Wilmington, NC""","""1JHK""","""202409""",49.0,378.75,67.0,2385.45,954.933333,954.933333,491.933333,491.933333,6533.533333,1345.5,1345.5,3.20625


port_name,dock_id,month,vessels,vessel_size_mean,calls,hrs_occupied,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean,hrs_in_port_waters_total,hrs_in_port_waters_mean,hrs_in_port_waters_median,utilization
str,str,str,u32,f64,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Albany Port District, NY""","""0PST""","""201803""",1,171.0,1,67.666667,67.666667,67.666667,0.0,0.0,73.283333,73.283333,73.283333,0.09095
"""Albany Port District, NY""","""0PST""","""201804""",1,189.0,1,165.6,165.6,165.6,0.0,0.0,172.833333,172.833333,172.833333,0.23
"""Albany Port District, NY""","""0PST""","""201806""",1,179.0,1,118.4,118.4,118.4,0.0,0.0,125.083333,125.083333,125.083333,0.164444
"""Albany Port District, NY""","""0PST""","""201807""",2,199.0,2,164.066667,82.033333,82.033333,0.0,0.0,200.683333,100.341667,100.341667,0.22052
"""Albany Port District, NY""","""0PST""","""201808""",3,192.666667,3,171.8,79.9,57.266667,0.0,0.0,350.9,116.966667,98.483333,0.230914


## NOTES on monthly stats
- As in calls_df, 0 values for hrs stats unexpected
    - docks that do not see any vessels do not appear in the underlying data since that's based on AIS moored statuses being matched within 500m of the dock
    - percentiles look reasonable so these may be edge cases with little effect on the stats
    - possibly drop call_ids with impossible 0 values before generating this dataframe?

#### Hours calc discussion

The current code first associates the call_id with the month in which the vessel arrived at dock, then counts total times for that call_id to that month. This results in some edge cases where hours stats far exceed the total hours in the month, as in the case that a vessel arrives at the dock and stays there for a very long period of time. 

This can be partially resolved by dropping statuses that are very long, which needs to be done anyway.
- what's the right strategy? set status duration to (the median for that dock? zero? 12hr?) and give an unknown status afterwards? 

It would be fully resolved by totaling monthly hrs (at dock or hrs utilized, for example) independently of call_id.  

In [73]:
#create annual dock stats dataframe
annual_docks_df = (
    calls_df
    #get year from docking time
    .with_columns(
        #extract year from docking time
        year = pl.col('time_arrival').dt.year()
    )
    #group by port dock and month
    .group_by(['port_name', 'dock_id', 'year'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')
    )
    #sort by port then month
    .sort(['port_name', 'year'])
)

#inspect
annual_docks_df.head()

port_name,dock_id,year,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,str,i32,u32,f64,u32,f64,f64,f64,f64
"""Albany Port District, NY""","""0RQB""",2018,3,144.333333,3,14.45,21.611111,0.0,0.0
"""Albany Port District, NY""","""0Y78""",2018,2,158.0,2,39.808333,39.808333,0.0,0.0
"""Albany Port District, NY""","""0Q86""",2018,3,136.333333,3,41.35,39.955556,0.0,0.0
"""Albany Port District, NY""","""0Y0U""",2018,11,140.333333,12,67.258333,81.781944,0.0,0.0
"""Albany Port District, NY""","""0Y0V""",2018,20,144.571429,21,66.3,70.378571,0.0,0.02619


In [74]:
#create annual port stats dataframe
annual_ports_df = (
    calls_df
    #get year from docking time
    .with_columns(
        #extract year from docking time
        year = pl.col('time_arrival').dt.year()
    )
    #group by port dock and month
    .group_by(['port_name', 'year'])
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')
    )
    #sort by port then month
    .sort(['port_name', 'year'])
)

#inspect
annual_ports_df.head()

port_name,year,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,i32,u32,f64,u32,f64,f64,f64,f64
"""Albany Port District, NY""",2018,60,153.227273,66,66.575,73.843939,0.0,0.008333
"""Albany Port District, NY""",2019,30,156.909091,33,69.366667,75.954545,0.0,0.018182
"""Albany Port District, NY""",2020,32,147.314286,35,67.566667,77.631905,0.0,0.0
"""Albany Port District, NY""",2021,45,157.574468,47,60.983333,74.12305,0.0,0.0
"""Albany Port District, NY""",2022,55,159.525424,59,74.483333,83.284746,0.0,1.532203


In [75]:
ports_alltime_df = (
    calls_df
    #group by port 
    .group_by('port_name')
    .agg(
        #count number of vessels
        vessels = pl.n_unique('mmsi'),
        #mean vessel size
        vessel_size_mean = pl.mean('vessel_size'),
        #count number of vessel calls
        calls = pl.n_unique('call_id'),
        #time at dock stats for each vessel in hours
        hrs_at_berth_median = pl.median('hrs_at_berth'),
        hrs_at_berth_mean = pl.mean('hrs_at_berth'),
        #time at anchor stats for each vessel visit in hours
        hrs_at_anchor_median = pl.median('hrs_at_anchor'),
        hrs_at_anchor_mean = pl.mean('hrs_at_anchor')
    )
    #sort by port
    .sort('port_name')
)
#inspect
ports_alltime_df

port_name,vessels,vessel_size_mean,calls,hrs_at_berth_median,hrs_at_berth_mean,hrs_at_anchor_median,hrs_at_anchor_mean
str,u32,f64,u32,f64,f64,f64,f64
"""Albany Port District, NY""",259,157.305785,363,65.8,77.066713,0.0,0.25303
"""Anacortes, WA""",255,219.480374,1070,37.033333,49.08553,0.025,31.896869
"""Baltimore, MD""",2727,214.74662,7692,34.108333,46.598613,0.0,16.116144
"""Beaumont, TX""",983,183.024031,1831,55.333333,67.308229,0.0,4.19564
"""Boston, MA""",293,296.619261,893,17.9,24.805636,0.0,2.366517
"""Bridgeport, CT""",6,199.042857,70,13.75,14.460476,0.0,2.101429
"""Brownsville, TX""",862,188.587173,1107,52.133333,61.481226,8.133333,36.078606
"""Calhoun Port Authority, TX""",497,160.48439,1057,26.766667,37.735762,12.633333,28.151766
"""Canaveral Port District, FL""",754,177.224839,1401,36.566667,45.669855,0.0,19.731751


In [76]:
#get point in time stats

#create point in time (pit) df to join stats to
pit_df = (
    main_lf
    .with_columns(
        date = pl.col('time').dt.date(),
        month = pl.col('time').dt.strftime('%Y%m')
    )
    .select('port_name', 'dock_id', 'month', 'date')
    .unique().collect()
)

for hour in range(0, 24):
    #create a time object for each hour
    hour_dt = pl.time(hour)
    #create a dataframe for each hour
    hour_df = (
        main_lf
        .with_columns(
            #get end of status time
            end_time = pl.col('time') + pl.col('status_duration'),
            #get date from time
            date = pl.col('time').dt.date(),
            #get month from time
            month = pl.col('time').dt.strftime('%Y%m')
        )
        #group by port dock and hour
        .group_by(['port_name', 'dock_id', 'month', 'date'])
        .agg(
            #number of vessels at dock at each hour
            vessels_at_dock = (
                #when moored at hour
                pl.when((pl.col('status')==5) & 
                        (hour_dt.is_between(pl.col('time').dt.time(), 
                                         pl.col('end_time').dt.time())))
                #then count the individual vessels
                .then(pl.col('mmsi'))
                .otherwise(pl.lit(None))
                .drop_nulls() #n_unique counts nulls as unique values
                .n_unique()
            ),
            #number of vessels at anchor at each hour
            vessels_at_anchor = (
                #when anchored at hour
                pl.when((pl.col('status')==1) & 
                        (hour_dt.is_between(pl.col('time').dt.time(), 
                                        pl.col('end_time').dt.time())))
                #then count the individual vessels
                .then(pl.col('mmsi'))
                .otherwise(pl.lit(None))
                .drop_nulls()
                .n_unique()
            )
        )
        .collect()
    )
    #join the hour dataframe to the main pit dataframe
    pit_df = (
        pit_df
        .join(hour_df, 
              on=['port_name', 'dock_id', 'month', 'date'], 
              how='left')
        #rename the columns to include the hour
        .rename({
            'vessels_at_dock': f'vessels_at_dock_{hour}',
            'vessels_at_anchor': f'vessels_at_anchor_{hour}'
        })
    )

#get port stats by month
pit_df = (
    pit_df
    #group by port and date
    .group_by(['port_name', 'month', 'date'])
    .agg(
        #sum the number of vessels at all docks at each hour
        cs.starts_with('vessels_at_dock_').sum(),
        #sum the number of vessels at anchor at each hour
        cs.starts_with('vessels_at_anchor_').sum()
    )
    #get the max at any hour
    .with_columns(
        #get max at dock at any hour
        vessels_at_dock_max = (
            pl.max_horizontal(cs.starts_with('vessels_at_dock_'))
        ),
        #get mean at dock any hour
        vessels_at_dock_mean = (
            pl.mean_horizontal(cs.starts_with('vessels_at_dock_'))
        ),
        #get max at anchor at any hour
        vessels_at_anchor_max = (
            pl.max_horizontal(cs.starts_with('vessels_at_anchor_'))
        ),
        #get mean at anchor any hour
        vessels_at_anchor_mean = (
            pl.mean_horizontal(cs.starts_with('vessels_at_anchor_'))
        )
    )
    #select the columns to keep
    .select(['port_name', 'month', 'date', 'vessels_at_dock_max', 
             'vessels_at_dock_mean', 'vessels_at_anchor_max',
             'vessels_at_anchor_mean'])
    #aggregate by month
    .group_by(['port_name', 'month'])
    .agg(
        #get max at anchor on any date during that month
        vessels_at_anchor_max = pl.max('vessels_at_anchor_max'),
        #get mean at anchor on any date during that month
        vessels_at_anchor_mean = pl.mean('vessels_at_anchor_mean'),
        #get max at dock on any date during that month
        vessels_at_dock_max = pl.max('vessels_at_dock_max'),
        #get mean at dock on any date during that month
        vessels_at_dock_mean = pl.mean('vessels_at_dock_mean'
        )
    )
)

In [77]:
#inspect
display(pit_df.describe())
pit_df.head()

statistic,port_name,month,vessels_at_anchor_max,vessels_at_anchor_mean,vessels_at_dock_max,vessels_at_dock_mean
str,str,str,f64,f64,f64,f64
"""count""","""5198""","""5198""",5198.0,5198.0,5198.0,5198.0
"""null_count""","""0""","""0""",0.0,0.0,0.0,0.0
"""mean""",null,null,0.781454,0.047546,2.375721,0.403511
"""std""",null,null,0.808571,0.089715,1.607373,0.400718
"""min""","""Albany Port District, NY""","""201801""",0.0,0.0,0.0,0.0
"""25%""",null,null,0.0,0.0,1.0,0.152778
"""50%""",null,null,1.0,0.013441,2.0,0.280864
"""75%""",null,null,1.0,0.056944,3.0,0.510057
"""max""","""Wilmington, NC""","""202409""",5.0,1.0125,11.0,3.58631


port_name,month,vessels_at_anchor_max,vessels_at_anchor_mean,vessels_at_dock_max,vessels_at_dock_mean
str,str,u32,f64,u32,f64
"""New Haven, CT""","""202101""",1,0.050926,2,0.287037
"""Brownsville, TX""","""202204""",1,0.059783,3,0.398551
"""Anacortes, WA""","""202010""",1,0.041667,2,0.138158
"""Galveston, TX""","""202008""",2,0.193548,4,0.524194
"""Baltimore, MD""","""202404""",1,0.035088,1,0.280702


### Max/Mean stats for vessels_at_dock 

- Current output seems far too low - e.g. Port of LA shows a max of 11 vessels at dock at any time since 2018; since there are 37 docks at LA we expect a max in the 20s or higher. 

In [78]:
pit_df.filter(pl.col('port_name')=='Port of Los Angeles, CA').sort('month')

port_name,month,vessels_at_anchor_max,vessels_at_anchor_mean,vessels_at_dock_max,vessels_at_dock_mean
str,str,u32,f64,u32,f64
"""Port of Los Angeles, CA""","""201801""",3,0.228495,5,1.254032
"""Port of Los Angeles, CA""","""201802""",3,0.267857,4,0.720238
"""Port of Los Angeles, CA""","""201803""",2,0.24328,7,1.294355
"""Port of Los Angeles, CA""","""201804""",3,0.309722,6,0.851389
"""Port of Los Angeles, CA""","""201805""",2,0.241935,3,0.875
"""Port of Los Angeles, CA""","""201806""",2,0.247222,6,1.001389
"""Port of Los Angeles, CA""","""201807""",3,0.217742,6,0.991935
"""Port of Los Angeles, CA""","""201808""",4,0.237903,4,0.876344
"""Port of Los Angeles, CA""","""201809""",2,0.294444,5,1.0
